<a href="https://colab.research.google.com/github/VISHAL-1-3/Resume-Parsing/blob/main/mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Import required libraries and verify installation
!pip install PyPDF2 # Install the missing 'PyPDF2' package
import PyPDF2
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install python-docx # install the missing 'docx' packageimport requests
import aiohttp

# Load spacy model
nlp = spacy.load("en_core_web_sm")

print("All libraries imported successfully!")

All libraries imported successfully!


In [44]:
import PyPDF2
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from typing import Dict, List, Tuple

class MultiDomainResumeAnalyzer:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

        # Specific skills database
        self.domain_skills_db = {
            'technology': {
                'programming': ['python', 'java', 'javascript', 'c++', 'ruby', 'php', 'scala', 'swift', 'kotlin',
                              'r', 'matlab', 'typescript', 'go', 'rust', 'perl', 'shell scripting'],
                'databases': ['sql', 'mongodb', 'postgresql', 'mysql', 'oracle', 'redis', 'cassandra',
                            'dynamodb', 'sqlite', 'neo4j', 'elasticsearch'],
                'frameworks': ['django', 'flask', 'react', 'angular', 'vue', 'spring', 'nodejs',
                             'express', 'fastapi', 'laravel', 'bootstrap', 'tailwind'],
                'cloud': ['aws', 'azure', 'gcp', 'docker', 'kubernetes', 'jenkins', 'terraform',
                         'ansible', 'cloud computing', 'serverless', 'microservices'],
                'machine_learning': ['tensorflow', 'pytorch', 'scikit-learn', 'machine learning', 'deep learning',
                                   'neural networks', 'nlp', 'computer vision', 'data mining', 'reinforcement learning',
                                   'pandas', 'numpy', 'matplotlib', 'seaborn', 'keras'],
                'tools': ['git', 'jira', 'confluence', 'bitbucket', 'maven', 'jenkins', 'docker',
                         'postman', 'swagger', 'jupyter', 'vscode', 'intellij', 'eclipse'],
                'office_tools': ['ms office', 'excel', 'word', 'powerpoint', 'outlook', 'access',
                               'google workspace', 'sharepoint', 'visio', 'project']
            },
            'finance': {
                'accounting': ['financial accounting', 'bookkeeping', 'balance sheet', 'income statement', 'cash flow',
                             'accounts payable', 'accounts receivable', 'tax accounting', 'audit'],
                'analysis': ['financial analysis', 'risk assessment', 'portfolio management', 'valuation',
                           'financial modeling', 'forecasting', 'budgeting', 'cost analysis'],
                'tools': ['excel', 'quickbooks', 'sap', 'bloomberg terminal', 'tableau', 'power bi',
                         'hyperion', 'sage', 'xero', 'factset'],
                'concepts': ['investment banking', 'private equity', 'hedge funds', 'derivatives', 'options trading',
                           'merger and acquisition', 'capital markets', 'fixed income', 'equity research'],
                'regulations': ['basel', 'sarbanes-oxley', 'dodd-frank', 'ifrs', 'gaap',
                              'sec regulations', 'aml', 'kyc', 'finra']
            },
            'hr': {
                'recruitment': ['talent acquisition', 'interviewing', 'job posting', 'candidate screening', 'onboarding',
                              'sourcing', 'employer branding', 'job description writing', 'recruitment marketing'],
                'systems': ['hris', 'ats', 'workday', 'successfactors', 'bamboo hr', 'oracle hcm',
                          'peoplesoft', 'kronos', 'ultipro', 'paycom'],
                'functions': ['benefits administration', 'compensation', 'employee relations', 'performance management',
                            'payroll', 'leave management', 'workplace safety', 'employee engagement'],
                'compliance': ['labor laws', 'employment law', 'osha', 'workplace safety', 'diversity and inclusion',
                             'ada compliance', 'eeoc', 'fmla', 'flsa', 'hipaa'],
                'development': ['training', 'leadership development', 'succession planning', 'career development',
                              'mentoring', 'coaching', 'skill assessment', 'performance evaluation']
            },
            'teaching': {
                'pedagogy': ['lesson planning', 'curriculum development', 'student assessment', 'classroom management'],
                'methods': ['differentiated instruction', 'project-based learning', 'blended learning', 'inquiry-based learning'],
                'tools': ['blackboard', 'canvas', 'moodle', 'google classroom', 'educational technology'],
                'skills': ['student engagement', 'classroom management', 'special education', 'esl', 'iep development'],
                'assessment': ['formative assessment', 'summative assessment', 'rubric development', 'standardized testing']
            },
            'marketing': {
                'digital': ['seo', 'sem', 'social media marketing', 'content marketing', 'email marketing'],
                'analytics': ['google analytics', 'marketing metrics', 'conversion optimization', 'a/b testing'],
                'tools': ['hubspot', 'mailchimp', 'google ads', 'facebook ads', 'adobe creative suite'],
                'skills': ['brand management', 'market research', 'campaign planning', 'copywriting'],
                'social': ['facebook', 'instagram', 'linkedin', 'twitter', 'tiktok']
            }
        }

        self.common_skills = {
            'soft_skills': ['leadership', 'communication', 'problem solving', 'organization'],
            'project_management': ['project planning', 'team management', 'event management']
        }

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            pdf_reader = PyPDF2.PdfReader(io.BytesIO(file.read()))
            for page in pdf_reader.pages:
                text += page.extract_text()
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return ""
        return text

    def extract_skills(self, text: str, domain: str) -> Dict[str, List[str]]:
        """Extract specific skills from text"""
        text = text.lower()
        found_skills = {'technical': [], 'soft': []}

        # Extract all specific technical skills
        for domain_cat in self.domain_skills_db.values():
            for category, skills in domain_cat.items():
                for skill in skills:
                    # Use word boundary check for more accurate matching
                    pattern = r'\b' + re.escape(skill) + r'\b'
                    if re.search(pattern, text):
                        found_skills['technical'].append(skill)

        # Extract soft skills
        for domain_cat in self.domain_skills_db.values():
            for skill in skills:
                pattern = r'\b' + re.escape(skill) + r'\b'
                if re.search(pattern, text):
                    found_skills['soft'].append(skill)

        # Remove duplicates and sort
        found_skills['technical'] = sorted(list(set(found_skills['technical'])))
        found_skills['soft'] = sorted(list(set(found_skills['soft'])))

        return found_skills

    def calculate_similarity(self, resume_text: str, job_desc: str) -> float:
        """Calculate similarity between resume and job description"""
        vectorizer = TfidfVectorizer()
        try:
            tfidf_matrix = vectorizer.fit_transform([resume_text, job_desc])
            return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except Exception as e:
            print(f"Error calculating similarity: {e}")
            return 0.0
    def recommend_courses(self, skill_gaps: List[str], domain: str) -> Dict[str, List[str]]:
      """Recommend courses based on skill gaps"""
      course_recommendations = {
        'technology': {
          'python': [
            'Complete Python Bootcamp: From Zero to Hero',
            'Python for Data Science and Machine Learning',
            'Advanced Python Programming',
            'Python Web Development with Django'
          ],
          'java': [
            'Java Programming Masterclass',
            'Spring Framework Complete Course',
            'Java Enterprise Edition Development',
            'Android App Development with Java'
          ],
          'javascript': [
            'Modern JavaScript from the Beginning',
            'Full Stack JavaScript Development',
            'React & Node.js Complete Developer Course',
            'Advanced JavaScript Concepts'
          ],
          'c++': [
            'C++ Programming from Scratch',
            'Game Development with C++',
            'Advanced C++ Programming',
            'Data Structures & Algorithms in C++'
          ],
          'typescript': [
            'TypeScript Complete Developer Guide',
            'Angular with TypeScript',
            'Enterprise TypeScript Development',
            'React & TypeScript Projects'
          ],
          'sql': [
            'Complete SQL Bootcamp',
            'Database Design & Management',
            'Advanced SQL for Data Analytics',
            'PostgreSQL Administration'
          ],
          'mongodb': [
            'MongoDB Complete Developer Guide',
            'NoSQL Database Development',
            'MERN Stack Development',
            'MongoDB Performance Optimization'
          ],
          'redis': [
            'Redis Fundamentals',
            'Caching Strategies with Redis',
            'Redis for Microservices',
            'High-Performance Systems with Redis'
          ],
          'react': [
            'React Complete Guide',
            'Advanced React and Redux',
            'React Native Mobile Development',
            'Full Stack React Development'
          ],
          'angular': [
            'Angular Complete Course',
            'Enterprise Angular Development',
            'Angular with TypeScript',
            'Full Stack Angular Projects'
          ],
          'django': [
            'Django Web Development',
            'Python & Django Full Stack',
            'Django REST Framework',
            'Django Projects Bootcamp'
          ],
          'aws': [
            'AWS Certified Solutions Architect',
            'AWS Developer Associate',
            'AWS DevOps Professional',
            'Serverless Applications on AWS'
          ],
          'azure': [
            'Microsoft Azure Fundamentals',
            'Azure Solutions Architect',
            'Azure DevOps Engineer',
            'Cloud Computing with Azure'
          ],
          'kubernetes': [
            'Kubernetes for Beginners',
            'Certified Kubernetes Administrator',
            'Container Orchestration',
            'Microservices with Kubernetes'
          ],
          'tensorflow': [
            'Deep Learning with TensorFlow',
            'TensorFlow Developer Certificate',
            'Machine Learning Projects',
            'Neural Networks with TensorFlow'
          ],
          'pytorch': [
            'PyTorch for Deep Learning',
            'Computer Vision with PyTorch',
            'Natural Language Processing',
            'Advanced AI Projects'
          ],
          'scikit-learn': [
            'Machine Learning with Scikit-Learn',
            'Data Science Bootcamp',
            'Predictive Analytics',
            'Statistical Learning'
          ]
        },
        'finance': {
          'financial accounting': [
            'Financial Accounting Fundamentals',
            'CPA Exam Preparation',
            'Corporate Accounting',
            'Advanced Financial Statements'
          ],
          'bookkeeping': [
            'Bookkeeping Basics',
            'QuickBooks Professional',
            'Small Business Accounting',
            'Advanced Bookkeeping'
          ],
          'audit': [
            'Internal Audit Fundamentals',
            'External Audit Practice',
            'Risk-Based Auditing',
            'IT Audit and Controls'
          ],
          'financial analysis': [
            'Financial Analysis Masterclass',
            'Investment Analysis',
            'Corporate Finance',
            'Financial Modeling'
          ],
          'risk assessment': [
            'Risk Management Professional',
            'Financial Risk Analysis',
            'Enterprise Risk Management',
            'Credit Risk Assessment'
          ]
        },
        'hr': {
          'talent acquisition': [
            'Strategic Talent Acquisition',
            'Recruitment and Selection',
            'HR Analytics for Recruiting',
            'Advanced Sourcing Techniques'
          ],
          'interviewing': [
            'Effective Interview Techniques',
            'Behavioral Interviewing',
            'Technical Recruitment',
            'Interview Assessment Methods'
          ],
          'hris': [
            'HRIS Implementation',
            'HR Analytics and Metrics',
            'HR Technology Management',
            'Digital HR Transformation'
          ],
          'workday': [
            'Workday HCM Administration',
            'Workday Implementation',
            'Workday Report Writing',
            'Workday Integration'
          ],
          'labor laws': [
            'Employment Law Essentials',
            'Labor Relations Management',
            'HR Compliance Certificate',
            'Workplace Investigations'
          ],
          'diversity and inclusion': [
            'DEI Strategy Development',
            'Inclusive Leadership',
            'Building Inclusive Workplaces',
            'Cultural Competence Training'
          ]
        }
      }


      recommendations = {}
      domain_courses = course_recommendations.get(domain, {})

      for skill in skill_gaps:
        # Try to find exact match first
        if skill in domain_courses:
            recommendations[skill] = domain_courses[skill]
        else:
            # Try to find partial matches
            for course_skill in domain_courses.keys():
                if (skill in course_skill) or (course_skill in skill):
                    recommendations[skill] = domain_courses[course_skill]
                    break

      return recommendations

def analyze_resume_and_job(resume_path: str, job_description: str) -> Dict:
    """Main analysis function"""
    analyzer = MultiDomainResumeAnalyzer()

    # Extract resume text
    resume_text = analyzer.extract_text_from_pdf(resume_path)
    if not resume_text:
        return {
            'error': 'Could not extract text from resume PDF'
        }

    # Detect domain based on job description keywords
    domain = detect_job_domain(job_description)

    # Extract skills using the detected domain
    resume_skills = analyzer.extract_skills(resume_text, domain)
    job_skills = analyzer.extract_skills(job_description, domain)

    # Calculate similarity
    similarity_score = analyzer.calculate_similarity(resume_text, job_description)

    # Identify skill gaps
    skill_gaps = list(set(job_skills['technical']) - set(resume_skills['technical']))

    # Get recommendations using the detected domain
    course_recommendations = analyzer.recommend_courses(skill_gaps, domain)

    return {
        'similarity_score': round(similarity_score * 100, 2),
        'resume_skills': resume_skills,
        'required_skills': job_skills,
        'skill_gaps': skill_gaps,
        'course_recommendations': course_recommendations,
        'detected_domain': domain
    }
def detect_job_domain(job_description: str) -> str:
    """
    Detect the domain of the job based on keywords in the description
    """
    # Convert to lowercase for case-insensitive matching
    description = job_description.lower()

    # Define domain keywords
    domain_keywords = {
        'technology': ['software', 'developer', 'programming', 'code', 'technical', 'engineer'],
        'finance': ['financial', 'accounting', 'finance', 'banking', 'investment'],
        'hr': ['hr', 'human resources', 'recruitment', 'hiring', 'talent'],
        'marketing': ['seo', 'marketing', 'campaign', 'digital', 'branding'],
        'teaching': ['teaching', 'pedagogy', 'curriculum', 'education', 'training']
    }

    # Count occurrences of keywords for each domain
    domain_scores = {}
    for domain, keywords in domain_keywords.items():
        score = sum(1 for keyword in keywords if keyword in description)
        domain_scores[domain] = score
    if all(score == 0 for score in domain_scores.values()):
        return "Error: Could not detect job domain from the description."
    # Return the domain with the highest score
    return max(domain_scores.items(), key=lambda x: x[1])[0]

def print_analysis(resume_path: str, job_description: str):
    """Print formatted analysis results"""
    results = analyze_resume_and_job(resume_path, job_description)

    if 'error' in results:
        print("\033[91m" + f"Error: {results['error']}" + "\033[0m")
        return

    print(f"\n{'-'*50}")
    print(f"📊 Similarity Score: {results['similarity_score']}%")
    print(f"🎯 Detected Domain: {results['detected_domain'].capitalize()}")
    print(f"{'-'*50}")

    print("\n📄 Skills Found in Resume:")
    print("\nTechnical Skills:")
    for skill in sorted(results['resume_skills']['technical']):
        print(f"  • {skill}")

    print("\nSoft Skills:")
    for skill in sorted(results['resume_skills']['soft']):
        print(f"  • {skill}")

    print(f"\n{'-'*50}")
    print("💼 Required Job Skills:")
    print("\nTechnical Skills:")
    for skill in sorted(results['required_skills']['technical']):
        print(f"  • {skill}")

    print("\nSoft Skills:")
    for skill in sorted(results['required_skills']['soft']):
        print(f"  • {skill}")

    if results['skill_gaps']:
        print(f"\n{'-'*50}")
        print("🎯 Skill Gaps and Recommended Courses:")
        for skill in sorted(results['skill_gaps']):
            print(f"\n\033[1mFor {skill}:\033[0m")
            if skill in results['course_recommendations']:
                for course in results['course_recommendations'][skill]:
                    print(f"  • {course}")
            else:
                print("  • No specific courses found for this skill")

    print(f"\n{'-'*50}")

In [37]:
resume_path = "/content/Vishal_Resume.pdf"
job_description = """We are looking for a Software Developer to build and implement functional programs. You will work with other Developers and Product Managers throughout the software development life cycle.

In this role, you should be a team player with a keen eye for detail and problem-solving skills. If you also have experience in Agile frameworks and popular coding languages (e.g. JavaScript), we’d like to meet you.

Your goal will be to build efficient programs and systems that serve user needs.

Responsibilities
Work with developers to design algorithms and flowcharts
Produce clean, efficient code based on specifications
Integrate software components and third-party programs
Verify and deploy programs and systems
Troubleshoot, debug and upgrade existing software
Gather and evaluate user feedback
Recommend and execute improvements
Create technical documentation for reference and reporting
Requirements and skills
Proven experience as a Software Developer, Software Engineer or similar role
Familiarity with Agile development methodologies
Experience with software design and development in a test-driven environment
Knowledge of coding languages (e.g. C++, Java, JavaScript) and frameworks/systems (e.g. AngularJS, Git)
Experience with databases and Object-Relational Mapping (ORM) frameworks (e.g. Hibernate)
Ability to learn new languages and technologies
Excellent communication skills
Resourcefulness and troubleshooting aptitude
Attention to detail
BSc/BA in Computer Science, Engineering or a related field"""
print_analysis(resume_path, job_description)


--------------------------------------------------
📊 Similarity Score: 24.25%
🎯 Detected Domain: Technology
--------------------------------------------------

📄 Skills Found in Resume:

Technical Skills:
  • excel
  • linkedin
  • machine learning
  • matplotlib
  • nlp
  • numpy
  • pandas
  • project
  • python
  • seaborn
  • sql
  • tableau

Soft Skills:
  • linkedin

--------------------------------------------------
💼 Required Job Skills:

Technical Skills:
  • git
  • java
  • javascript

Soft Skills:

--------------------------------------------------
🎯 Skill Gaps and Recommended Courses:

For git:
  • No specific courses found for this skill

For java:
  • Java Programming Masterclass
  • Spring Framework Complete Course
  • Java Enterprise Edition Development
  • Android App Development with Java

For javascript:
  • Modern JavaScript from the Beginning
  • Full Stack JavaScript Development
  • React & Node.js Complete Developer Course
  • Advanced JavaScript Concepts

----

In [38]:
import pickle

# Example model data (replace with your actual model)
model = {
    "technology": ['software', 'developer', 'programming', 'code', 'technical', 'engineer'],
    "finance": ['financial', 'accounting', 'finance', 'banking', 'investment'],
    "hr": ['hr', 'human resources', 'recruitment', 'hiring', 'talent'],
    "marketing": ['seo', 'marketing', 'campaign', 'digital', 'branding'],
    "teaching": ['teaching', 'pedagogy', 'curriculum', 'education', 'training']
}

# File path to save the model
file_path = "modal.pkl"

# Save the model to a .pkl file
with open(file_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved to {file_path}")


Model saved to modal.pkl


In [43]:
print("PyPDF2:", PyPDF2.__version__)
print("spaCy:", spacy.__version__)
print("pandas:", pd.__version__)
import sklearn # Import the sklearn library
print("scikit-learn:", sklearn.__version__) # Access the version using __version__
print("re:", "Built-in, no version applicable")

PyPDF2: 3.0.1
spaCy: 3.7.5
pandas: 2.2.2
scikit-learn: 1.6.0
re: Built-in, no version applicable


In [46]:
import pickle

# Assuming `model` is your trained model
with open('modal.pkl', 'wb') as file:
    pickle.dump(model, file)


In [47]:
import pickle

with open('modal.pkl', 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data)


<class 'dict'>
{'technology': ['software', 'developer', 'programming', 'code', 'technical', 'engineer'], 'finance': ['financial', 'accounting', 'finance', 'banking', 'investment'], 'hr': ['hr', 'human resources', 'recruitment', 'hiring', 'talent'], 'marketing': ['seo', 'marketing', 'campaign', 'digital', 'branding'], 'teaching': ['teaching', 'pedagogy', 'curriculum', 'education', 'training']}
